In [17]:
import glob
import os
import re
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

In [2]:
!pwd

/home/user


In [2]:
import sys  
sys.path.insert(0, '../home/user')

In [3]:
import modeling

2025-11-20 13:19:34.118347: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/home/user/anaconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-20 13:19:40,368	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
!ls /home/user

 anaconda3			       modeling.ipynb
 Anaconda3-2024.10-1-Linux-x86_64.sh   modeling.py
 Anaconda3-2025.06-0-Linux-x86_64.sh   Musique
 Bureau				       Public
 collect_hw_vm.sh		       snap
 datasets			       Téléchargements
 Documents			      'testing modularisation.ipynb'
 Images				       Untitled.ipynb
 Modèles			       Vidéos


In [4]:
!jupyter nbconvert --to python /home/user/modeling.ipynb

[NbConvertApp] Converting notebook /home/user/modeling.ipynb to python
[NbConvertApp] Writing 9660 bytes to /home/user/modeling.py


In [5]:
from dotenv import load_dotenv
import os

load_dotenv("/home/user/.env")
DagsHub_username = os.getenv("DagsHub_username")
DagsHub_token=os.getenv("DagsHub_token")

print(os.getenv("DagsHub_username"))
print(os.getenv("DagsHub_token"))

emnaaS
ac026871943a805c04cfaa7af4cf1a0734c83779


In [6]:
import os
os.environ['MLFLOW_TRACKING_USERNAME']= DagsHub_username
os.environ["MLFLOW_TRACKING_PASSWORD"] = DagsHub_token

In [7]:
import dagshub
dagshub.init(repo_owner='emnaaS', repo_name='kmeans_repo', mlflow=True)

Accessing as emnaaS

Initialized MLflow to track repo "emnaaS/kmeans_repo"

Repository emnaaS/kmeans_repo initialized!

In [4]:
from modeling import load_image_dataset
images, labels = load_image_dataset(
    image_dir="/home/user/datasets/splits/batch_1000",
    labels_file="/home/user/datasets/trainLabels_cropped.csv",
    max_images=100)

Loaded 100 images with labels


In [8]:
from modeling import prepare_data
images_flat, labels_flat = prepare_data(images, labels)

images shape (100, 196608)
labels shape (100,)


In [9]:
from modeling import K_Means_Distributed

In [14]:
import os
import time
import json
import numpy as np
import mlflow
import ray

# Start MLflow run
with mlflow.start_run():

    # MLflow: log parameters
    mlflow.log_param("n_clusters", 3)
    mlflow.log_param("max_iter", 500)
    mlflow.log_param("num_workers", 2)
    mlflow.log_param("num_images", len(images_flat))
    
    # Ray memory configuration
    os.environ['RAY_memory_usage_threshold'] = '0.97'
    os.environ['RAY_memory_monitor_refresh_ms'] = '250'
    num_features = images_flat.shape[1]
    mlflow.log_param("num_features", num_features)

    # Shutdown previous Ray cluster if needed
    if ray.is_initialized():
        ray.shutdown()

    # Init Ray
    ray.init(num_cpus=2)
    print(ray.cluster_resources())

    # Train model
    start_time = time.time()
    kmeans_dist = K_Means_Distributed(
        n_clusters=3,
        max_iter=500,
        num_workers=2
    )

    kmeans_dist.fit(images_flat)
    end_time = time.time()

    training_time = end_time - start_time
    print(f"Training time = {training_time:.2f} seconds")

    # MLflow: log training time
    mlflow.log_metric("training_time_seconds", training_time)

    # Retrieve results from your KMeans class
    centroids = kmeans_dist.cluster_centers_
    num_iterations = kmeans_dist.number_of_iter
    cluster_labels = kmeans_dist.labels_

    # MLflow: Log iteration count
    mlflow.log_metric("iterations", num_iterations)

    # Compute cluster sizes
    unique, counts = np.unique(cluster_labels, return_counts=True)
    cluster_size_dict = {int(k): int(v) for k, v in zip(unique, counts)}

    # MLflow: Log size of each cluster
    for cluster_id, size in cluster_size_dict.items():
        mlflow.log_metric(f"cluster_{cluster_id}_size", size)

    # Save centroids
    np.save("centroids.npy", centroids)
    mlflow.log_artifact("centroids.npy")

    # Save cluster report
    with open("cluster_sizes.json", "w") as f:
        json.dump(cluster_size_dict, f, indent=4)

    mlflow.log_artifact("cluster_sizes.json")

    print("\n✓ Logged centroids, iterations, and cluster sizes to MLflow successfully.\n")


2025-11-20 13:33:48,264	INFO worker.py:2013 -- Started a local Ray instance.


{'node:192.168.62.206': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 9293700710.0, 'memory': 21685301658.0}
Data splitting & Ray.put: 0.053s

=== Training Started ===

--- Iteration 1 ---
(Worker pid=290543) Worker - Distance computation: 0.406s | Aggregation: 0.036s | Total: 0.490s
Ray.get (collection): 0.905s
Result aggregation: 0.008s
Centroid update: 0.006s
Convergence check: 0.005s | Center shift: 267.837549

--- Iteration 2 ---
Ray.get (collection): 0.482s
Result aggregation: 0.006s
Centroid update: 0.004s
Convergence check: 0.003s | Center shift: 72.524473

--- Iteration 3 ---
Ray.get (collection): 0.471s
Result aggregation: 0.004s
Centroid update: 0.004s
Convergence check: 0.003s | Center shift: 40.830633

--- Iteration 4 ---
Ray.get (collection): 0.432s
Result aggregation: 0.004s
Centroid update: 0.004s
Convergence check: 0.003s | Center shift: 8.630597

--- Iteration 5 ---
Ray.get (collection): 0.432s
Result aggregation: 0.004s
Centroid update: 0.005

In [1]:
import mlflow

In [13]:
!pip show mlflow

Name: mlflow
Version: 2.22.2
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: https://mlflow.org
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.

                                Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the

In [14]:
import sys
print(sys.executable)


/home/user/anaconda3/envs/myenv/bin/python


In [16]:
!/home/user/anaconda3/envs/myenv/bin/python -m pip install mlflow

  Using cached mlflow-3.6.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_skinny-3.6.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_tracing-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached alembic-1.17.2-py3-none-any.whl.metadata (7.2 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached huey-2.5.4-py3-none-any.whl.metadata (4.6 kB)
  Using cached sqlalchemy-2.0.44-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.5 kB)
  Using cached databricks_sdk-0.73.0-py3-none-any.whl.metadata (40 kB)
  Using cached fastapi-0.121.3-py3-none-any.whl.metadata (30 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached openteleme